# Backbone for convolutional feature map: Resnet

In [1]:
import numpy as np 
import keras
from keras import layers
from keras.layers import LeakyReLU, Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D,Conv2DTranspose, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.initializers import glorot_uniform
from keras.regularizers import l2, l1
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.models import model_from_json

import pydot
import graphviz
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import pickle

from IPython.display import SVG
from keras.utils import np_utils 
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
# from keras.utils.training_utils import multi_gpu_model

import matplotlib.pyplot as plt

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)
from keras.utils import to_categorical
from keras import optimizers
from keras import regularizers


import tensorflow as tf
import os

/home/suhaila/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def identity_block(X, filters, stage, block):
    """
    Implementation of the identity block where the input skips over one convolutional layer
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (3, 3), strides = (1,1), padding = 'same', name = conv_name_base + '2a', kernel_initializer ='he_normal',kernel_regularizer=l2(1e-4))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # Second component of main path (≈2 lines)
    X = Conv2D(filters = F2, kernel_size = (3, 3), strides = (1,1), padding = 'same', name = conv_name_base + '2c', kernel_initializer ='he_normal',kernel_regularizer=l2(1e-4))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X_shortcut,X])
    X =  Activation('relu')(X)
    
    
    return X

In [3]:
def convolutional_block(X,filters, stage, block):
    """
    Implementation of the convolutional block in ResNet to reduce feature map size, increase number of filters
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (3, 3), strides = (2,2), name = conv_name_base + '2a',padding='same', kernel_initializer ='he_normal',kernel_regularizer=l2(1e-4))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path (≈2 lines)
    X = Conv2D(F2, (3, 3), strides = (1,1), name = conv_name_base + '2c',padding='same', kernel_initializer ='he_normal',kernel_regularizer=l2(1e-4))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    #Shortcut path to match dimensions
    X_shortcut = Conv2D(F2, (1, 1), strides = (2,2), name = conv_name_base + '1',padding='same' ,kernel_initializer ='he_normal',kernel_regularizer=l2(1e-4))(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X_shortcut,X])
    X =  Activation('relu')(X)
    
    return X

In [4]:
def Network_multitask(input_shape = (256, 256, 3)):
    """
    Implementation of the ResNet
    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    X = X_input
    
    #Bounding box regression + class score branch 
    
    # Initial Convolutional layer
    X = Conv2D(16, (3, 3), strides = (1, 1), name = 'conv1', padding = 'same', kernel_initializer='he_normal',kernel_regularizer=l1(1e-4))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X =  Activation('relu')(X)

    # Stage 1 with three resnet blocks 
    X = identity_block(X,[16, 16], stage=1, block='a')
    X = identity_block(X, [16, 16], stage=1, block='b')
    X = identity_block(X, [16, 16], stage=1, block='c')

    # Stage 2 : 1 convolutional and 2 identity blocks
    X = convolutional_block(X, filters = [32, 32], stage = 2, block='a')
    X = identity_block(X, [32, 32], stage=2, block='b')
    X = identity_block(X, [32,32], stage=2, block='c')


    # Stage 3: 1 convolutional and 2 identity blocks
    X = convolutional_block(X, filters = [64, 64], stage = 3, block='a')
    X = identity_block(X, [64,64], stage=3, block='b')
    X = identity_block(X, [64,64], stage=3, block='c')
    
#     Stage 4: 1 convolutional and 2 identity blocks
#     X = convolutional_block(X, filters = [128, 128], stage = 4, block='a')
#     X = identity_block(X, [128,128], stage=4, block='b')
#     X = identity_block(X, [128,128], stage=4, block='c')
    
    X = Conv2D(filters = 32, kernel_size = (3, 3), strides = (2,2), padding = 'same', kernel_initializer ='he_normal',kernel_regularizer=l2(1e-4))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters = 5, kernel_size = (3, 3), strides = (1,1), padding = 'same', kernel_initializer ='he_normal',kernel_regularizer=l2(1e-4))(X)
    X = BatchNormalization(axis = 3)(X)
    bounding_box_score = Activation('sigmoid', name = 'output1')(X)
     
    
    X = bounding_box_score
    
    #Segmentation Branch 
    
    X = Conv2D(filters = 32, kernel_size = (3, 3), strides = (2,2), padding = 'same', kernel_initializer ='he_normal',kernel_regularizer=l2(1e-4))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    
    X = Conv2DTranspose(filters = 32, kernel_size = (2, 2), strides=(2, 2), padding='valid', use_bias=True, kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    
    X = Conv2DTranspose(filters = 25, kernel_size = (2, 2), strides=(2, 2), padding='valid', use_bias=True, kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    
    X = Conv2DTranspose(filters = 10, kernel_size = (2, 2), strides=(2, 2), padding='valid', use_bias=True, kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    
    X = Conv2DTranspose(filters = 5, kernel_size = (2, 2), strides=(2, 2), padding='valid', use_bias=True, kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    
    X = Conv2DTranspose(filters = 1, kernel_size = (1, 1), strides=(1, 1), padding='valid', use_bias=True, kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(X)
    X = BatchNormalization(axis = 3)(X)
    
    segmentation = Activation('sigmoid', name = 'output2')(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = [bounding_box_score, segmentation], name='Network_multitask')

    return model

In [5]:
#Number of gpus
n_gpu = 0

#Single GPU Training
if n_gpu <=1:
    model = Network_multitask(input_shape = (256, 256, 3)) 

#Multi GPU Training
else:
    with tf.device("/device:CPU:0"):
        model = Network_multitask(input_shape = (256, 256, 3))
        #Recommended so that the weights are stored and updated in
        #the CPU so as to avoid problems with weight sharing between GPUs 
   
    model = multi_gpu_model(model, gpus=n_gpu)
        
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 256, 256, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 256, 256, 16) 64          conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 256, 256, 16) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
res1a_bran

In [6]:
plot_model(model, to_file='model1.png', show_shapes=True, show_layer_names=False)

In [7]:
#Saving model 
model.save('model1.h5')